In [235]:
import glob
import os
import subprocess
import numpy as np
import pandas as pd
import math

from tqdm import tqdm
from pathlib import Path

# What's happening here?

In this notebook we will read in all the csv files and compile it to useful information. 
We will end up with comprehensive CSV files containing all usefull information. 

Add to this list:
- Responses
- RT
- Testmoment
- Ppt number
- Total time taken


Make list
Check for all instances with that name
if exists: wCam
- get that
else:
- get other



# Get Data

In [2]:
# Declare all directories:

data_dir = Path('Z:/shares/ghepmk_data/2020_Kappen_PMS')

# Create all relevant directories
date_dir = "24082021/"
data_all_dir = os.path.join(data_dir, date_dir)
data_A_dir = os.path.join(data_dir, Path('10001'))
data_B_dir = os.path.join(data_dir, Path('10002'))

# TESTMOMENT A

In [276]:
os.chdir(data_A_dir)
ACsv = glob.glob("csv/" + "*ParticipantNo*" + "*.csv", recursive = True) # get all csv's in directory
ACsvID = []
CSVy_n = []
substring = "wCAM" # This is only present in some files but is prefered

for index, item in enumerate(ACsv):
    ACsvID.append(ACsv[index].split("data_{ParticipantNo}_")[1][0:3])
    if substring in ACsv[index]:
        CSVy_n.append(True)
    else:
        CSVy_n.append(False)
        
ACsvID = set(ACsvID)

# demographics = pd.read_csv("Z:/shares/ghepmk_data/2020_Kappen_PMS/24082021/cleanData.csv", encoding = "ISO-8859-1")
demographics = pd.read_csv(data_all_dir + "cleanData.csv", encoding = "ISO-8859-1")

In [284]:
exclude = ['203', # Double moment A
           '262', # Double moment A
           '285', # Double moment A (both wCAM)
           '334', # Double moment A
           '312', # Double moment A
           '361', # Double moment A, but second time moment A also did moment B (2 days later).
           '531', # Double moment A
           '661', # Double moment A, but second time moment A also did moment B.
           '667', # Double moment A, but second time moment A also did moment B. Only 4 days between moment A and B, figure out # i might have been wrong based on video. Almost month later
           
           '105', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '106', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '107', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '111', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '118', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '119', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '123', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '124', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '125', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '127', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '778', # No demographics present. Last known participant number is 677. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '999', # No demographics present. Last known participant number is 677. So probably a testing round
           '998', # No demographics present. Last known participant number is 677. So probably a testing round
           
           '407', # Double demographics entry (seems like they first filled out with wrong mentrual duration) - figure this out
           
           '373', # Has no proper experiment order - check in R file
           
           '66_', # Weird participant number, let's see if we can figure out what the correct one would've been here -> change name of csv/webm
           '6_1', # Very hacky, because if the particpiant ends in '6' and date starts with '1' this will also get triggered. Figure out how this ends up in the csv file and try to solve
           '&à&', # Weird stuff, shouldn't be here
           '11_', # Probably the same as the others, check it out
          ]

In [288]:
index = 0
for pptnum in tqdm(ACsvID): # Loop through all file ID's
#     print("Processing: " + pptnum)
    if pptnum not in exclude: # Some participants messed up, so skip those. See earlier cell for descriptions of each
        index += 1
        # Find right file to load and read_csv in
        indices = [i for i, s in enumerate(ACsv) if pptnum in s and "wCAM" not in s] # Check if there is a file WITHOUT wCAM because that is preferred (saved later)
        if len(indices) == 1: # if one, that is perfect, read it
            data = pd.read_csv(ACsv[indices[0]])
        if len(indices) > 1: # if more than one, something is going wrong, figure out
            print("Too many .csv files for ppt " + pptnum + ".")
            break
        elif len(indices) == 0: # if none, probably only saved before video, so check wCAM
            indices = [i for i, s in enumerate(ACsv) if pptnum in s and "wCAM" in s] # Find instances where files exist WITH wCAM
            if len(indices) == 1: # if one, that is perfect, read it
                data = pd.read_csv(ACsv[indices[0]])
            if len(indices) > 1: # if more than one, something is going wrong, figure out
                print("Too many wCAM.csv files for ppt " + pptnum + ".")
                break
            elif len(indices) == 0: # If none, something weird is happening, as it should have appeared here then
                print("No wCAM.csv files for ppt " + pptnum + ".")
                break

        #  Now check whether they completed the experiment
        if not data['stimulus'].str.contains('beep.wav').any(): # if beep.wav is present (final thing in the experiment) they finished
            print("ppt " + pptnum + " did not finish the experiment.")
            break

        # Get data in Demographics file
    #     demoindex = [i for i, s in enumerate(demographics['participantNo'] == int(pptnum)) if s][0]
        demoindex = np.where(demographics['participantNo'] == int(pptnum))[0]
        if len(demoindex) > 1: # Too many entries for this ppt
            print("Too many demographics entries for ppt " + pptnum + ".")
            break
        elif len(demoindex) == 0:
            print("No demographics entries for ppt " + pptnum + ".")
            break
        elif len(demoindex) == 1:
            # Create indices for the trial selections
            indices = [i for i, s in enumerate(data['trial_type']) if "image-keyboard-response" in s]
            indices2 = [x+1 for x in indices]
            stimNum = len(indices) # to add constant values to dataframe

            # Get correct PSS and BSRI values
            statementAB = demographics['Order'][demoindex].apply(str) == "A-B"
            statementBA = demographics['Order'][demoindex].apply(str) == "B-A"

            if statementAB.bool() == True: # If order is A-B, and this is moment A, then A is folliculair
                PSS = demographics['folliculairPSS'][demoindex].values[0]
                BSRI = demographics['folliculairBSRI'][demoindex].values[0]
            elif statementBA.bool() == True: # If order is B-A, and this is moment A, then A is luteal
                PSS = demographics['luteaalPSS'][demoindex].values[0]
                BSRI = demographics['luteaalBSRI'][demoindex].values[0]
            else:
                print("ppt " + pptnum + " does not have a valid experiment order.")

            df = pd.DataFrame(list(zip([pptnum] * stimNum, [demographics['Age'][demoindex].values[0]] * stimNum, [demographics['FirstMenstrual'][demoindex].values[0]] * stimNum, [demographics['MenstrualDuration'][demoindex].values[0]] * stimNum,
                                       [demographics['Order'][demoindex].values[0]] * stimNum, ['A'] * stimNum, [demographics['allSymptoms'][demoindex].values[0]] * stimNum, [demographics['allDisturbance'][demoindex].values[0]] * stimNum, [demographics['allRRS'][demoindex].values[0]] * stimNum,
                                       [demographics['DASS.Total'][demoindex].values[0]] * stimNum, [demographics['DASS.Stress'][demoindex].values[0]] * stimNum, [demographics['DASS.Anxiety'][demoindex].values[0]] * stimNum, [demographics['DASS.Depresh'][demoindex].values[0]] * stimNum,
                                       [demographics['PMSScore'][demoindex].values[0]] * stimNum, [demographics['Contraception'][demoindex].values[0]] * stimNum, [PSS] * stimNum, [BSRI] * stimNum,
                                       data['stimulusShort'][indices], data['valence'][indices2], data['arousal'][indices2], data['rt'][indices2], )),
                           columns =['ID', 'Age', 'FirstMenstrual', 'MenstrualDuration', 
                                     'Order', 'Moment', 'allSymptoms', 'allDistrubance', 'allRRS', 
                                     'DASS_Total', 'DASS_Stress', 'DASS_Anxiety', 'DASS_Depression', 
                                     'PMSScore', 'Contraception', 'PSS', 'BSRI',
                                     'Stimulus', 'Valence', 'Arousal', 'rt'])
        if index == 1:
            dataset = df
        else:
            dataset = pd.concat([dataset, df], ignore_index=True)
    #         break
# dataset
datasetA = dataset

100%|████████████████████████████████████████████████████████████████████████████████| 389/389 [00:45<00:00,  8.48it/s]


In [289]:
datasetA

,ID,Age,FirstMenstrual,MenstrualDuration,Order,Moment,allSymptoms,allDistrubance,allRRS,DASS_Total,...,DASS_Anxiety,DASS_Depression,PMSScore,Contraception,PSS,BSRI,Stimulus,Valence,Arousal,rt
0,527,29,14,30.0,A-B,A,26,5,25,29,...,11,11,0,Natural,34.0,216.0,1710.0,100.0,100.0,4952.175
1,527,29,14,30.0,A-B,A,26,5,25,29,...,11,11,0,Natural,34.0,216.0,6313.0,0.0,0.0,5280.585
2,527,29,14,30.0,A-B,A,26,5,25,29,...,11,11,0,Natural,34.0,216.0,2057.0,100.0,100.0,5425.835
3,527,29,14,30.0,A-B,A,26,5,25,29,...,11,11,0,Natural,34.0,216.0,5600.0,100.0,100.0,4619.280
4,527,29,14,30.0,A-B,A,26,5,25,29,...,11,11,0,Natural,34.0,216.0,9592.0,50.0,0.0,3912.950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7937,266,39,12,5.0,A-B,A,24,7,45,41,...,15,15,0,Natural,27.0,1.0,2057.0,75.0,39.0,11781.675
7938,266,39,12,5.0,A-B,A,24,7,45,41,...,15,15,0,Natural,27.0,1.0,2165.0,74.0,43.0,7015.245
7939,266,39,12,5.0,A-B,A,24,7,45,41,...,15,15,0,Natural,27.0,1.0,4658.0,86.0,60.0,21709.245
7940,266,39,12,5.0,A-B,A,24,7,45,41,...,15,15,0,Natural,27.0,1.0,9921.0,14.0,53.0,13433.070


# TESTMOMENT B

In [293]:
os.chdir(data_B_dir)
BCsv = glob.glob("csv/" + "*ParticipantNo*" + "*.csv", recursive = True) # get all csv's in directory
BCsvID = []
CSVy_n = []
substring = "wCAM" # This is only present in some files but is prefered

for index, item in enumerate(BCsv):
    BCsvID.append(BCsv[index].split("data_{ParticipantNo}_")[1][0:3])
    if substring in BCsv[index]:
        CSVy_n.append(True)
    else:
        CSVy_n.append(False)
        
BCsvID = set(BCsvID)

# demographics = pd.read_csv("Z:/shares/ghepmk_data/2020_Kappen_PMS/24082021/cleanData.csv", encoding = "ISO-8859-1")
demographics = pd.read_csv(data_all_dir + "cleanData.csv", encoding = "ISO-8859-1")

In [342]:
exclude = ['100', # No demographics present. First known participant number is 129.
           '106', # No demographics present. First known participant number is 129. 
           '119', # No demographics present. First known participant number is 129. 
           '125', # No demographics present. First known participant number is 129.
           '126', # No demographics present. First known participant number is 129.
           '127', # No demographics present. First known participant number is 129.
           '778', # No demographics present. Last known participant number is 677. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '789', # No demographics present. Last known participant number is 677. So probably wrong participant number. Figure out and then -> change name of csv/webm
           
           
           '350', # Double moment B, but also 1 moment A. Two moment B's 5 days apart. Weird. Figure out
           '141', # Double moment B - both wCAM
           '405', # Double moment B
           
           '407', # Double demographics entry (seems like they first filled out with wrong mentrual duration) - figure this out
           
           '590', # Kicked out for now, because '4590' is also present (which is wrong). When 4590 is resolved, 590 can come back
           '667', # Kicked out because also '6677' present. So someone did something wrong here. Probably need to kik out 6677 (rename?) -> figure out
           
           
           '14_', # Weird stuff
           '_28', # Weird stuff
           '66_', # Weird stuff
           
           '987', # Trial run probably
           '995', # No demographics, Trial run probably
           '996', # Trial run probably
           '997', # Trial run probably
           '999', # Trial run probably
          ]

In [343]:
index = 0
for pptnum in tqdm(BCsvID): # Loop through all file ID's
#     print("Processing: " + pptnum)
    if pptnum not in exclude: # Some participants messed up, so skip those. See earlier cell for descriptions of each
        index += 1
        # Find right file to load and read_csv in
        indices = [i for i, s in enumerate(BCsv) if pptnum in s and "wCAM" not in s] # Check if there is a file WITHOUT wCAM because that is preferred (saved later)
        if len(indices) == 1: # if one, that is perfect, read it
            data = pd.read_csv(BCsv[indices[0]])
        if len(indices) > 1: # if more than one, something is going wrong, figure out
            print("Too many .csv files for ppt " + pptnum + ".")
            break
        elif len(indices) == 0: # if none, probably only saved before video, so check wCAM
            indices = [i for i, s in enumerate(BCsv) if pptnum in s and "wCAM" in s] # Find instances where files exist WITH wCAM
            if len(indices) == 1: # if one, that is perfect, read it
                data = pd.read_csv(BCsv[indices[0]])
            if len(indices) > 1: # if more than one, something is going wrong, figure out
                print("Too many wCAM.csv files for ppt " + pptnum + ".")
                break
            elif len(indices) == 0: # If none, something weird is happening, as it should have appeared here then
                print("No wCAM.csv files for ppt " + pptnum + ".")
                break

        #  Now check whether they completed the experiment
        if not data['stimulus'].str.contains('beep.wav').any(): # if beep.wav is present (final thing in the experiment) they finished
            print("ppt " + pptnum + " did not finish the experiment.")
            break

        # Get data in Demographics file
    #     demoindex = [i for i, s in enumerate(demographics['participantNo'] == int(pptnum)) if s][0]
        demoindex = np.where(demographics['participantNo'] == int(pptnum))[0]
        if len(demoindex) > 1: # Too many entries for this ppt
            print("Too many demographics entries for ppt " + pptnum + ".")
            break
        elif len(demoindex) == 0:
            print("No demographics entries for ppt " + pptnum + ".")
            break
        elif len(demoindex) == 1:
            # Create indices for the trial selections
            indices = [i for i, s in enumerate(data['trial_type']) if "image-keyboard-response" in s]
            indices2 = [x+1 for x in indices]
            stimNum = len(indices) # to add constant values to dataframe

            # Get correct PSS and BSRI values
            statementAB = demographics['Order'][demoindex].apply(str) == "A-B"
            statementBA = demographics['Order'][demoindex].apply(str) == "B-A"

            if statementAB.bool() == True: # If order is A-B, and this is moment B, then A is luteal
                PSS = demographics['luteaalPSS'][demoindex].values[0]
                BSRI = demographics['luteaalBSRI'][demoindex].values[0]
            elif statementBA.bool() == True: # If order is B-A, and this is moment B, then A is folliculair
                PSS = demographics['folliculairPSS'][demoindex].values[0]
                BSRI = demographics['folliculairBSRI'][demoindex].values[0]
            else:
                print("ppt " + pptnum + " does not have a valid experiment order.")

            df = pd.DataFrame(list(zip([pptnum] * stimNum, [demographics['Age'][demoindex].values[0]] * stimNum, [demographics['FirstMenstrual'][demoindex].values[0]] * stimNum, [demographics['MenstrualDuration'][demoindex].values[0]] * stimNum,
                                       [demographics['Order'][demoindex].values[0]] * stimNum, ['B'] * stimNum, [demographics['allSymptoms'][demoindex].values[0]] * stimNum, [demographics['allDisturbance'][demoindex].values[0]] * stimNum, [demographics['allRRS'][demoindex].values[0]] * stimNum,
                                       [demographics['DASS.Total'][demoindex].values[0]] * stimNum, [demographics['DASS.Stress'][demoindex].values[0]] * stimNum, [demographics['DASS.Anxiety'][demoindex].values[0]] * stimNum, [demographics['DASS.Depresh'][demoindex].values[0]] * stimNum,
                                       [demographics['PMSScore'][demoindex].values[0]] * stimNum, [demographics['Contraception'][demoindex].values[0]] * stimNum, [PSS] * stimNum, [BSRI] * stimNum,
                                       data['stimulusShort'][indices], data['valence'][indices2], data['arousal'][indices2], data['rt'][indices2], )),
                           columns =['ID', 'Age', 'FirstMenstrual', 'MenstrualDuration', 
                                     'Order', 'Moment', 'allSymptoms', 'allDistrubance', 'allRRS', 
                                     'DASS_Total', 'DASS_Stress', 'DASS_Anxiety', 'DASS_Depression', 
                                     'PMSScore', 'Contraception', 'PSS', 'BSRI',
                                     'Stimulus', 'Valence', 'Arousal', 'rt'])
        if index == 1:
            dataset = df
        else:
            dataset = pd.concat([dataset, df], ignore_index=True)
    #         break
# dataset
datasetB = dataset

100%|████████████████████████████████████████████████████████████████████████████████| 379/379 [00:26<00:00, 14.52it/s]


In [344]:
dataset = pd.concat([datasetA, datasetB], ignore_index=True)
dataset

,ID,Age,FirstMenstrual,MenstrualDuration,Order,Moment,allSymptoms,allDistrubance,allRRS,DASS_Total,...,DASS_Anxiety,DASS_Depression,PMSScore,Contraception,PSS,BSRI,Stimulus,Valence,Arousal,rt
0,527,29,14,30.0,A-B,B,26,5,25,29,...,11,11,0,Natural,36.0,585.0,7238.0,100.0,100.0,5057.400
1,527,29,14,30.0,A-B,B,26,5,25,29,...,11,11,0,Natural,36.0,585.0,1722.0,100.0,81.0,3255.700
2,527,29,14,30.0,A-B,B,26,5,25,29,...,11,11,0,Natural,36.0,585.0,9584.0,0.0,99.0,7877.700
3,527,29,14,30.0,A-B,B,26,5,25,29,...,11,11,0,Natural,36.0,585.0,9594.0,18.0,79.0,4079.500
4,527,29,14,30.0,A-B,B,26,5,25,29,...,11,11,0,Natural,36.0,585.0,1750.0,100.0,100.0,3996.300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7849,226,42,13,27.0,B-A,B,24,9,31,38,...,13,12,0,Natural,29.0,432.0,5982.0,60.0,76.0,5137.500
7850,226,42,13,27.0,B-A,B,24,9,31,38,...,13,12,0,Natural,29.0,432.0,9584.0,35.0,67.0,4460.690
7851,226,42,13,27.0,B-A,B,24,9,31,38,...,13,12,0,Natural,29.0,432.0,7481.0,69.0,27.0,5781.430
7852,226,42,13,27.0,B-A,B,24,9,31,38,...,13,12,0,Natural,29.0,432.0,9920.0,22.0,70.0,5446.735


In [350]:
os.chdir(os.path.join(data_dir,date_dir))
dataset.to_csv('allPMSdata.csv')